In [1]:
# dependencies
import pandas as pd
import numpy as np
import csv

In [2]:
csv_schools_path = "Resources/schools_complete.csv"
csv_students_path ="Resources/students_complete.csv"

schools_complete_df = pd.read_csv(csv_schools_path)
students_complete_df = pd.read_csv(csv_students_path)
schools_complete_df.shape

(15, 5)

In [3]:
#clean up school dataframe:
schools_complete_df=schools_complete_df.drop('School ID',axis=1)
schools_clean_df = schools_complete_df.sort_values(by=['school_name'])
schools_clean_df = schools_clean_df.set_index('school_name')

#Rename column header for consistency
students_clean_df = students_complete_df.rename(columns={'Student ID': 'student_id'})

In [4]:
#Merge both csv files
pycityschools_df = pd.merge(schools_clean_df,students_clean_df,on="school_name")


In [5]:
#Calculate number of schools:
school_count = len(pycityschools_df.school_name.unique())

#Calculate number of students 
    #used student_id to ensure no duplication of names
student_count = len(pycityschools_df.student_id.unique())


#Calculate total budget using original school csv file:
#school_budget=schools_clean_df.budget.unique()
total_budget = schools_clean_df.budget.sum(axis=0)

#Caculate ave scores for math and reading
ave_math_score = round(pycityschools_df.math_score.mean(),2)
ave_reading_score = round(pycityschools_df.reading_score.mean(),2)

In [6]:
#Pass/Fail determination for Math:

#Count of students passing math:
math_pass_list= pycityschools_df.loc[pycityschools_df['math_score']>=70]['math_score'].count()
percentage_pass_math = round((math_pass_list/student_count*100),2)

#Count of students passing reading:
reading_pass_list = pycityschools_df.loc[pycityschools_df['reading_score']>=70]['reading_score'].count()
percentage_pass_reading = round((reading_pass_list/student_count*100),2)

#Count of overll passing - must pass both reading & math:
overall_pass_list = pycityschools_df[(pycityschools_df['math_score']>=70) & (pycityschools_df['reading_score']>=70)]['student_id'].count()
percentage_pass_overall = round((overall_pass_list/student_count*100),2)


In [7]:
#Key metrics table - district level
district_data =[{'Total Schools':school_count,
                 'Total Students': student_count,
                 'Total Budget': total_budget, 
                 'Ave Math Score':ave_math_score,
                 'Ave Reading Score':ave_reading_score,
                 '% Passing Math':percentage_pass_math,
                 '% Passing Reading':percentage_pass_reading,
                 '% Overall Passing': percentage_pass_overall}]
data_district_summary_df=pd.DataFrame(district_data)
data_district_summary_df

,Total Schools,Total Students,Total Budget,Ave Math Score,Ave Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.99,81.88,74.98,85.81,65.17


In [8]:
school_data_df = pycityschools_df.set_index('school_name').groupby(['school_name'])


In [9]:
#School Summary:

#Create dataframe showing type of school:
school_type=school_data_df['type'].first()
school_type_df=pd.DataFrame(school_type)

#Create dataframe showing students/sschool:
school_student=school_data_df['student_id'].count()
school_student_df=pd.DataFrame(school_student)

#Merge first 2 dataframes together:
school_summary_df=pd.merge(school_type,school_student_df,how='left',on='school_name')

#Create dataframe showing school budget:
school_budget=school_data_df['budget'].first()
school_budget_df=pd.DataFrame(school_budget)

#Merge dataframe with prior merge:
school_summary_df=pd.merge(school_summary_df,school_budget_df,how='left',on='school_name')

#Calculate budget per student per school & create dataframe:
student_budget=school_budget/school_student
student_budget_df=pd.DataFrame(student_budget)

#Merge dataframe with prior merge:
school_summary_df=pd.merge(school_summary_df,student_budget_df,how='left',on='school_name')

#Calculate ave math score per school & create dataframe:
school_ave_math=round(school_data_df['math_score'].mean(),2)
school_ave_math_df=pd.DataFrame(school_ave_math)

#Merge dataframe with prior merge:
school_summary_df=pd.merge(school_summary_df,school_ave_math_df,how='left',on='school_name')

#Calculate ave reading score per school & create dataframe:
school_ave_reading=round(school_data_df['reading_score'].mean(),2)
school_ave_reading_df=pd.DataFrame(school_ave_reading)

#Merge dataframe with prior merge:
school_summary_df=pd.merge(school_summary_df,school_ave_reading_df,how='left',on='school_name')

#Calculate % of students passing math & create a dataframe:
school_math_perc=round((pycityschools_df[pycityschools_df['math_score']>=70].groupby('school_name')['student_id'].count())/(school_student)*100,2)
school_math_perc_df=pd.DataFrame(school_math_perc)

#Merge dataframe with prior merge:
school_summary_df=pd.merge(school_summary_df,school_math_perc_df,how='left',on='school_name')

#Calculate % of students passing reading & create a dataframe:
school_reading_perc=round((pycityschools_df[pycityschools_df['reading_score']>=70].groupby('school_name')['student_id'].count())/(school_student)*100,2)
school_reading_perc_df=pd.DataFrame(school_reading_perc)

#Merge dataframe with prior merge:
school_summary_df=pd.merge(school_summary_df,school_reading_perc_df,how='left',on='school_name')

#Calculate % of students passing overall & create a dataframe:
overall_pass_count=pycityschools_df[(pycityschools_df['math_score']>=70) & (pycityschools_df['reading_score']>=70)].groupby('school_name')['student_id'].count()
overall_pass_perc=round((overall_pass_count/school_student)*100,2)
overall_pass_perc_df=pd.DataFrame(overall_pass_perc)

#Final merge:
school_summary_df=pd.merge(school_summary_df,overall_pass_perc_df,how='left',on='school_name')

#Clean up display
school_summary_df.columns=['School Type','Total Students','Total School Budget','Per Student Budget','Ave Math Score','Ave Reading Score','% Passing Math', '% Passing Reading', '% Overall Passing']
school_summary_df.rename_axis('School Name', inplace = True)

school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Ave Math Score,Ave Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Charter,1858,1081356,582.0,83.06,83.98,94.13,97.04,91.33
Figueroa High School,District,2949,1884411,639.0,76.71,81.16,65.99,80.74,53.20
Ford High School,District,2739,1763916,644.0,77.10,80.75,68.31,79.30,54.29
Griffin High School,Charter,1468,917500,625.0,83.35,83.82,93.39,97.14,90.60
Hernandez High School,District,4635,3022020,652.0,77.29,80.93,66.75,80.86,53.53
Holden High School,Charter,427,248087,581.0,83.80,83.81,92.51,96.25,89.23
Huang High School,District,2917,1910635,655.0,76.63,81.18,65.68,81.32,53.51
Johnson High School,District,4761,3094650,650.0,77.07,80.97,66.06,81.22,53.54


In [10]:
#Top 5 Performing Schools:
school_summary_df.sort_values(by='% Overall Passing',ascending=False).head(5)


,School Type,Total Students,Total School Budget,Per Student Budget,Ave Math Score,Ave Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.06,83.98,94.13,97.04,91.33
Thomas High School,Charter,1635,1043130,638.0,83.42,83.85,93.27,97.31,90.95
Griffin High School,Charter,1468,917500,625.0,83.35,83.82,93.39,97.14,90.60
Wilson High School,Charter,2283,1319574,578.0,83.27,83.99,93.87,96.54,90.58
Pena High School,Charter,962,585858,609.0,83.84,84.04,94.59,95.95,90.54


In [11]:
#Bottom 5 Performing Schools:
school_summary_df.sort_values(by='% Overall Passing',ascending=True).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Ave Math Score,Ave Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.84,80.74,66.37,80.22,52.99
Figueroa High School,District,2949,1884411,639.0,76.71,81.16,65.99,80.74,53.20
Huang High School,District,2917,1910635,655.0,76.63,81.18,65.68,81.32,53.51
Hernandez High School,District,4635,3022020,652.0,77.29,80.93,66.75,80.86,53.53
Johnson High School,District,4761,3094650,650.0,77.07,80.97,66.06,81.22,53.54


In [12]:
#Re-index dataframe and group by grade level:
scores_by_grade_df = pycityschools_df.set_index('grade').groupby(['grade'])

In [13]:
#Math scores by Grade Level:
math_scores_grade = scores_by_grade_df['math_score'].mean()
math_scores_grade_df=pd.DataFrame(math_scores_grade)
math_scores_grade_df

,math_score
grade,
10th,78.941483
11th,79.083548
12th,78.993164
9th,78.935659


In [14]:
#Reading scores by Grade Level:
reading_scores_grade = scores_by_grade_df['reading_score'].mean()
reading_scores_grade_df=pd.DataFrame(reading_scores_grade)
reading_scores_grade_df

,reading_score
grade,
10th,81.874410
11th,81.885714
12th,81.819851
9th,81.914358


In [20]:
#Binning school budgets for: Scores by School Spending:
school_spending_df=pycityschools_df
spending_bins=[0,999999,1249999,1999999,4000000]
spending_names=["<$1,000,000", " <$1,250,000","<$2,000,000",">$2,000,000"]
school_spending_df['Budget Limits']=pd.cut(school_spending_df['budget'],bins=spending_bins,labels=spending_names)
school_spending_df

spending_school_df = school_spending_df.set_index('Budget Limits').groupby(['Budget Limits'])
spending_school_df.head()

,school_name,type,size,budget,student_id,student_name,gender,grade,reading_score,math_score
Budget Limits,,,,,,,,,,
">$2,000,000",Bailey High School,District,4976,3124928,17871,Blake Martin,M,9th,75,59
">$2,000,000",Bailey High School,District,4976,3124928,17872,Kathryn Kane,F,12th,84,58
">$2,000,000",Bailey High School,District,4976,3124928,17873,Richard Haas,M,11th,79,86
">$2,000,000",Bailey High School,District,4976,3124928,17874,Frank Marsh,M,9th,71,89
">$2,000,000",Bailey High School,District,4976,3124928,17875,Charles Goodman Jr.,M,9th,90,61
"<$1,250,000",Cabrera High School,Charter,1858,1081356,16013,Olivia Short,F,11th,94,94
"<$1,250,000",Cabrera High School,Charter,1858,1081356,16014,Kerry Jones,F,9th,98,97
"<$1,250,000",Cabrera High School,Charter,1858,1081356,16015,Bruce Jackson,M,12th,78,83
"<$1,250,000",Cabrera High School,Charter,1858,1081356,16016,Amy Green,F,12th,89,92


In [31]:
#Scores by School Spending

#Calcuation for ave math score by spending bins & create dataframe
ave_math_by_spending=spending_school_df['math_score'].mean()
ave_math_by_spending_df=pd.DataFrame(ave_math_by_spending)

#Calcuation for ave reading score by spending bins & create dataframe
ave_reading_by_spending = spending_school_df['reading_score'].mean()
ave_reading_by_spending_df = pd.DataFrame(ave_reading_by_spending)

#Count of students by spending bins:
spending_student_count=spending_school_df['student_id'].count()

#Calculations of Math passing percentages by spending bins:
spending_math_perc=round((school_spending_df[school_spending_df['math_score']>=70].groupby('Budget Limits')['student_id'].count())/(spending_student_count)*100,2)
spending_math_perc_df=pd.DataFrame(spending_math_perc)

#Calculations of Reading passing percentages by spending bins:
spending_reading_perc=round((school_spending_df[school_spending_df['reading_score']>=70].groupby('Budget Limits')['student_id'].count())/(spending_student_count)*100,2)
spending_reading_perc_df=pd.DataFrame(spending_math_perc)
spending_reading_perc_df

#Calculations of Overall passing percentages by spending bins:
spending_pass_count = school_spending_df[(school_spending_df['math_score']>=70) & (school_spending_df['reading_score']>=70)].groupby('Budget Limits')['student_id'].count()
spending_pass_perc=round((spending_pass_count/spending_student_count)*100,2)
spending_pass_perc_df=pd.DataFrame(spending_pass_perc)

#Merge ave Math & Reading DataFrames
spending_summary_df=pd.merge(ave_math_by_spending_df,ave_reading_by_spending_df,how='left',on='Budget Limits')
spending_summary_df=pd.merge(spending_summary_df, spending_math_perc_df,how='left',on='Budget Limits')
spending_summary_df=pd.merge(spending_summary_df,spending_reading_perc_df,how='left',on='Budget Limits')
spending_summary_df=pd.merge(spending_summary_df,spending_pass_perc_df,how='left',on='Budget Limits')
spending_summary_df.columns=['Ave Math Score', 'Ave Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']

#Display table
spending_summary_df

,Ave Math Score,Ave Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Budget Limits,,,,,
"<$1,000,000",83.583479,83.893245,93.66,93.66,90.37
"<$1,250,000",83.377091,83.878650,93.66,93.66,90.63
"<$2,000,000",78.164034,81.654758,72.34,72.34,61.40
">$2,000,000",77.070764,80.928365,66.47,66.47,53.72
